In [1]:
import pickle
import numpy as np
from hdmm import templates, workload, error, more_templates
%load_ext autoreload
%autoreload 2

In [2]:
W = pickle.load(open('household_wkld.pckl', 'rb'))
dom = (2,9,2,7,8,24,4,2)
#temp = templates.Marginals(dom)

#A, loss = temp.restart_optimize(W, 10)

In [3]:
queries = [[] for _ in range(8)]

for kron in W.matrices:
    for sub, q in zip(kron.matrices, queries):
        q.append(sub.dense_matrix())

queries = [np.vstack(q) for q in queries]

In [4]:
for i in range(8):
    Q = np.unique(queries[i], axis=0)
    idx1 = Q.sum(axis=1)>1
    idx2 = ~Q.all(axis=1)
    print(Q.astype(np.uint8))

[[0 1]
 [1 0]
 [1 1]]
[[0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 1 1]
 [0 0 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 0 0 0]
 [1 1 1 1 1 1 1 1 1]]
[[0 1]
 [1 0]
 [1 1]]
[[0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0]
 [0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0]
 [0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [1 1 1 1 1 1 1]]
[[0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 1 1 1 1 1 1]
 [0 1 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1]]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1

In [5]:
W5 = workload.EkteloMatrix(queries[5].astype(np.float))
pid = templates.PIdentity(24, 24)

rows = W5.dense_matrix()
rows = np.unique(rows, axis=0)
rows = rows[rows.sum(axis=1) > 1]

#row = more_templates.RowWeighted(rows)
#A, loss = pid.restart_optimize(W5, 1000)
#B, loss2 = row.restart_optimize(W5, 100)
#print(loss)

899.659140765667


In [6]:
new = []

for kron in W.matrices:
    subs = [workload.EkteloMatrix(WW.dense_matrix().astype(float)) for WW in kron.matrices]
    W1, W2, W3, W4, W5, W6, W7, W8 = subs
    A = workload.Kronecker([W1, W2, W3, W4, W5, W7, W8])
    B = W6
    new.append(workload.Kronecker([A, B]))
    
new = workload.VStack(new)

In [ ]:
dom1 = (2,9,2,7,8,4,2)
T1 = templates.Marginals(dom1)
T2 = templates.PIdentity(20, 24)
T3 = templates.Identity(24)
T4 = templates.BestTemplate([T2, T3])
_templates = [T1, T4]

workloads = [[Q for Q in K.matrices] for K in new.matrices]

k = len(workloads)
d = len(workloads[0])
C = np.ones((d,k))

for i in range(d):
    for j in range(k):
        C[i,j] = workloads[j][i].gram().trace()
for _ in range(10):
    for i in range(d):
        #print(C)
        temp = _templates[i]
        cs = np.sqrt(C.prod(axis=0) / C[i])
        What = workload.VStack([c*Ws[i] for c, Ws in zip(cs, workloads)])
        A, loss = temp.restart_optimize(What, 1000)
        for j in range(k):
            #print(error.expected_error(workloads[j][i], A))
            C[i,j] = error.expected_error(workloads[j][i], A)

    loss = C.prod(axis=0).sum()
    print(np.sqrt(loss / W.shape[0]))